In [1]:
from __future__ import print_function, division
#Import needed packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#from onnx_tf.backend import prepare
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.autograd import Variable
#from pytorch2keras.converter import pytorch_to_keras
import matplotlib.pyplot as plt
import time
import os
import copy
import csv
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from datetime import datetime
from sklearn import metrics
from pathlib import Path
plt.ion()   # interactive mode

In [2]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [3]:
# Mnist dataset fpr Lenet
batch_size = 128
T = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])
train_data = torchvision.datasets.MNIST('mnist_data', train=True, download=True, transform=T)
val_data = torchvision.datasets.MNIST('mnist_data', train=False, download=True, transform=T)

trainloader = torch.utils.data.DataLoader(train_data, batch_size = batch_size)
valloader = torch.utils.data.DataLoader(val_data, batch_size = batch_size)

/opt/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
import onnx
import onnxruntime
import coremltools
import time

Using TensorFlow backend.


In [5]:
onnx_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/onnx/'
coreml_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/coremltools/'
error_path = '/Volumes/Cisco/Fall2021/onnx-exchange/miss-classification/v2/'

In [22]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
def model_scores(y_test, test_predict):
    correct_ = np.sum(y_test == test_predict)#128
    accuracy  = correct_ #*100./np.sum(y_test == y_test)#
    return accuracy
def get_miss_classification(y1, y2):
    miss_perc_val_original_coreml = 0
    try:
        np.testing.assert_array_equal(y1, y2)
    except Exception as e:
        miss_perc_val_original_coreml = 1
    return miss_perc_val_original_coreml #, precision2, recall2, f12

In [23]:
def to_onnx(i, x, y, batch_size):
    
    # Input to the model
    #device_reset = cuda.get_current_device()
    #device_reset.reset()
    #x.cuda()
    #print("converting for batch: ", i)
    
    #torch.random.manual_seed(42)
    #x = torch.randn(10000, 3, 32, 32, requires_grad=True)
    traced_model = torch.jit.trace(model, x)
    
    ### Original Model
    since_1 = time.time()
    torch_out = model(x)
    inference_time_original = time.time() - since_1
    y0 = torch.argmax(torch_out,1)
    accuracy_original = model_scores(to_numpy(y), to_numpy(y0))
    #miss_test_original = get_miss_classification(to_numpy(y), to_numpy(y0))
    # ONNX Model
    
    t_elapsed_2 = time.time() - since_1
    #since_1 = time.time()
    #onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    #load_time_onnx = time.time() - since_1
    #onnx.checker.check_model(onnx_model)
    #size_onnx = os.path.getsize(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    #def to_numpy(tensor):
    #    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    #ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    since_1 = time.time()
    ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
    ort_outs = ort_session.run(None, ort_inputs)
    inference_time_onnx = time.time() - since_1
    # compare ONNX Runtime and PyTorch results
    #print("\n*********\n\n")
    #time_diff = t_elapsed_0+t_elapsed_1, t_elapsed_2, t_elapsed_3
    
    ####### Mis-classification ONNX ######################################
    ort_outs_tensor = torch.from_numpy(ort_outs[0]) 
    y2 = torch.argmax(ort_outs_tensor,1)
    #y2 = to_categorical(np.argmax(ort_outs[0], 1), num_classes = 10)
    #correct_onnx = np.sum(y2 == y)
    accuracy_onnx = model_scores(to_numpy(y), to_numpy(y2))
    miss_original_onnx = get_miss_classification(to_numpy(y0), to_numpy(y2))
    ####### End of mis-classification ONNX ###################################### 
    
    
    ## CoreML
    
    ## Converting the coremltool
    #since_1 = time.time()
    # Using image_input in the inputs parameter:
    # Convert to Core ML using the Unified Conversion API.
    #coreml_model = coremltools.convert(traced_model,inputs=[coremltools.TensorType(shape=x.shape)])
    #t_conversion_time_coreml = time.time() - since_1
    #since_1 = time.time()
    #coreml_model.save(coreml_path+framework+'/{}/{}-v2.mlmodel'.format(model_short_name, model_name))
    #t_saving_time_coreml = time.time() - since_1
    
    
    #print(name_1)

    #size_coreml = os.path.getsize(coreml_path+framework+'/{}/{}-v2.mlmodel'.format(model_short_name, model_name))
    
    
    #since_1 = time.time()
    #coreml_model = coremltools.models.MLModel(coreml_path+framework+"/{}/{}-v2.mlmodel".format(model_short_name, model_name))
    #load_time_coreml = time.time() - since_1
    
    #spec = coreml_model.get_spec()
    #coreml_model = coremltools.models.MLModel(spec)
    name_1 = coreml_model.get_spec().description.input[0].name
    since_1 = time.time()
    output_dict_test = coreml_model.predict({name_1:to_numpy(x)})
    inference_time_coreml = time.time() - since_1
    ####### Mis-classification coreML ######################################
    coreml_array_output = output_dict_test[list(output_dict_test.keys())[0]]
    output_tensor = torch.from_numpy(coreml_array_output) 
    y3 = torch.argmax(output_tensor,1)
    #correct_coreml = np.sum(to_numpy(y3) == to_numpy(y))
    accuracy_coreml = model_scores(to_numpy(y3), to_numpy(y))
    miss_original_coreml = get_miss_classification(to_numpy(y0), to_numpy(y3))
    ####### End of mis-classification coreML ######################################
    list_val = [accuracy_original, accuracy_onnx, accuracy_coreml, miss_original_onnx, miss_original_coreml,inference_time_original, inference_time_onnx, inference_time_coreml]
    return list_val

In [24]:
def _lets_convert(data,  data_writer_run, batch_size): # for cifar10 etc
    since = time.time()
    accuracy_original = 0.0
    accuracy_onnx = 0.0
    accuracy_coreml = 0.0
    #miss_test_original = 0
    miss_original_onnx = 0
    #miss_test_onnx = 0
    miss_original_coreml = 0
    #miss_test_coreml = 0
    
    inference_time_original = 0.0
    inference_time_onnx = 0.0
    inference_time_coreml = 0.0
    
    total_ = 0.0
    total_datasets = 0
    for i, (images, labels) in enumerate(data):
        #print(images.shape)
        list_val = to_onnx(i, images,labels, batch_size)
        #if list_val != None:
        if i%50 == 0:
            print(i, list_val)
        accuracy_original += list_val[0]
        accuracy_onnx += list_val[1]
        accuracy_coreml += list_val[2]

        miss_original_onnx += list_val[3]
        miss_original_coreml += list_val[4]

        inference_time_original += list_val[5]
        inference_time_onnx += list_val[6]
        inference_time_coreml += list_val[7]
        total_ += np.sum(to_numpy(labels) == to_numpy(labels))
        total_datasets += labels.shape[0]
        if i == 1000:
            break
    time_elapsed = time.time() - since
    print('Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60) )
    data_writer_run.writerow([model_short_name,framework, training_id, model_name, round(accuracy_original*100/total_,2), round(accuracy_onnx*100/total_,2), round(accuracy_coreml*100/total_,2),  round(miss_original_onnx*100/total_datasets,2),  round(miss_original_coreml*100/total_datasets,2),'', inference_time_original/total_datasets, inference_time_onnx/total_datasets, inference_time_coreml/total_datasets, '',
                              '{:.0f}m {:.0f}s'.format((inference_time_original/total_datasets) // 60, (inference_time_original/total_datasets) % 60),
                              '{:.0f}m {:.0f}s'.format((inference_time_onnx/total_datasets) // 60, (inference_time_onnx/total_datasets) % 60),
                              '{:.0f}m {:.0f}s'.format((inference_time_coreml/total_datasets) // 60, (inference_time_coreml/total_datasets) % 60),
                              '{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)])

In [26]:
import pandas as pd 

#data_file_run = open(error_path+framework+"/{}/runtime_miss-classification_{}.csv".format(model_short_name,model_name), mode='w', newline='',
#                                  encoding='utf-8')
#data_writer_run = csv.writer(data_file_run, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#data_writer_run.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','runtime','original_load_time', 'original_infererence_time', 'runtime_load_time', 
#                          'runtime_inference_time',  'miss_classified_original_runtime_percentage','',  'encoded_miss_classified_original_runtime_percentage','encoded_miss_classified_original_test_runtime_percentage', '', 'accuracy_original', 'accuracy_runtime'])
valloader = torch.utils.data.DataLoader(val_data, batch_size = 1)
for i, (images, labels) in enumerate(valloader):
    images_ = images
    labels_ = labels 
    break
shape_ = images_.shape
for round_ in [2,3,4,5,6,7,8,9,10]: #,2,3,4,5,6,7,8,9,10
    training_id = round_
    model_short_name = 'lenet5'
    framework = 'pytorch'
    path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/{}/{}/'.format(framework, model_short_name)

    # Model class must be defined somewhere
    since_0 = time.time()
    model_name = 'torch_lenet5-mnist_2021-11-01_{}'.format(training_id)
    #model_name = 'torch_lenet5-mnist_2021-11-01_{}'.format(training_id)
    model = torch.load(path+model_name+'.pth', map_location=torch.device('cpu'))
    #resnet50_model.eval()
    t_elapsed_0 = time.time() - since_0
    size0 = os.path.getsize(path+model_name+'.pth')
    size0
    
    since_1 = time.time()
    onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    load_time_onnx = time.time() - since_1
    onnx.checker.check_model(onnx_model)
    size_onnx = os.path.getsize(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    #def to_numpy(tensor):
    #    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    
    
    # Convert to Core ML using the Unified Conversion API.
    traced_model = torch.jit.trace(model, images_)
    coreml_model = coremltools.convert(traced_model,inputs=[coremltools.TensorType(shape=images_.shape)])
    #t_conversion_time_coreml = time.time() - since_1
    
    #since_1 = time.time()
    #coreml_model = coremltools.models.MLModel(coreml_path+framework+"/{}/{}-v2.mlmodel".format(model_short_name, model_name))
    #load_time_coreml = time.time() - since_1
    
    if not os.path.exists(error_path+framework):
        Path(error_path+framework).mkdir(parents=True, exist_ok=True)
    
    print('round: ', round_)
    flag = 0
    if not os.path.exists(error_path+framework+"/runtime_accuracy_{}.csv".format(model_short_name)):
        data_file_run = open(error_path+framework+"/runtime_accuracy_{}.csv".format(model_short_name), mode='w', newline='', encoding='utf-8')
    else:
        data_file_run = open(error_path+framework+"/runtime_accuracy_{}.csv".format(model_short_name), mode='a+', newline='', encoding='utf-8')
        flag = 1
    data_writer_run = csv.writer(data_file_run, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    if flag == 0: 
        data_writer_run.writerow(['model','framework', 'training_id', 'model_full', 'accuracy_original', 'accuracy_onnx', 'accuracy_coreml', 'miss_original_onnx', 'miss_original_coreml','', 'inference_time_original', 'inference_time_onnx', 'inference_time_coreml', '', 'inference_time_original2', 'inference_time_onnx2', 'inference_time_coreml2', 'overral_time'])
    # dataloaders
    batch_size = 1 
    valloader = torch.utils.data.DataLoader(val_data, batch_size = batch_size)
    _lets_convert(valloader, data_writer_run, batch_size)
    
    data_file_run.close()

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 45/45 [00:00<00:00, 1256.94 ops/s]


round:  2
0 [1, 1, 1, 0, 0, 0.001950979232788086, 0.00040078163146972656, 0.007871866226196289]
50 [1, 1, 1, 0, 0, 0.0014102458953857422, 0.0003657341003417969, 0.0022869110107421875]
100 [1, 1, 1, 0, 0, 0.0012359619140625, 0.00032520294189453125, 0.002343893051147461]
150 [1, 1, 1, 0, 0, 0.0012798309326171875, 0.0003190040588378906, 0.0037009716033935547]
200 [1, 1, 1, 0, 0, 0.0012578964233398438, 0.0003666877746582031, 0.005804777145385742]
250 [1, 1, 1, 0, 0, 0.0010879039764404297, 0.00032782554626464844, 0.006124734878540039]
300 [1, 1, 1, 0, 0, 0.0013489723205566406, 0.0003151893615722656, 0.009440898895263672]
350 [1, 1, 1, 0, 0, 0.0013880729675292969, 0.00037217140197753906, 0.005349874496459961]
400 [1, 1, 1, 0, 0, 0.0013780593872070312, 0.00032401084899902344, 0.004287004470825195]
450 [1, 1, 1, 0, 0, 0.0012688636779785156, 0.0003571510314941406, 0.012959957122802734]
500 [1, 1, 1, 0, 0, 0.0015239715576171875, 0.0003361701965332031, 0.0036478042602539062]
550 [1, 1, 1, 0, 0, 0

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 45/45 [00:00<00:00, 1624.13 ops/s]


round:  3
0 [1, 1, 1, 0, 0, 0.001422882080078125, 0.000415802001953125, 0.005249977111816406]
50 [1, 1, 1, 0, 0, 0.0031061172485351562, 0.0003628730773925781, 0.0042324066162109375]
100 [1, 1, 1, 0, 0, 0.011851072311401367, 0.003049135208129883, 0.03148913383483887]
150 [1, 1, 1, 0, 0, 0.0013887882232666016, 0.00035691261291503906, 0.003994941711425781]
200 [1, 1, 1, 0, 0, 0.004968881607055664, 0.0003631114959716797, 0.003940105438232422]
250 [1, 1, 1, 0, 0, 0.0012938976287841797, 0.0003380775451660156, 0.0037958621978759766]
300 [1, 1, 1, 0, 0, 0.001043081283569336, 0.00029540061950683594, 0.005434751510620117]
350 [1, 1, 1, 0, 0, 0.0017120838165283203, 0.00039386749267578125, 0.00913381576538086]
400 [1, 1, 1, 0, 0, 0.001386880874633789, 0.00027179718017578125, 0.003668069839477539]
450 [1, 1, 1, 0, 0, 0.0025551319122314453, 0.0004379749298095703, 0.043478965759277344]
500 [1, 1, 1, 0, 0, 0.0011179447174072266, 0.0003178119659423828, 0.002206087112426758]
550 [1, 1, 1, 0, 0, 0.001648

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 45/45 [00:00<00:00, 1279.94 ops/s]


...

round:  8
0 [1, 1, 1, 0, 0, 0.0010387897491455078, 0.0004138946533203125, 0.00421905517578125]
50 [1, 1, 1, 0, 0, 0.0008869171142578125, 0.0002899169921875, 0.004060029983520508]
100 [1, 1, 1, 0, 0, 0.0009469985961914062, 0.000270843505859375, 0.0038712024688720703]
150 [1, 1, 1, 0, 0, 0.0014681816101074219, 0.0002570152282714844, 0.0022771358489990234]
200 [1, 1, 1, 0, 0, 0.0008749961853027344, 0.00024080276489257812, 0.003515958786010742]
250 [1, 1, 1, 0, 0, 0.0013861656188964844, 0.000308990478515625, 0.0037622451782226562]
300 [1, 1, 1, 0, 0, 0.0017061233520507812, 0.0002791881561279297, 0.004772186279296875]
350 [1, 1, 1, 0, 0, 0.0009222030639648438, 0.0002560615539550781, 0.0034112930297851562]
400 [1, 1, 1, 0, 0, 0.0010530948638916016, 0.0003161430358886719, 0.007542133331298828]
450 [1, 1, 1, 0, 0, 0.001188039779663086, 0.0005550384521484375, 0.0023679733276367188]
500 [1, 1, 1, 0, 0, 0.0028879642486572266, 0.00041174888610839844, 0.011742830276489258]
550 [1, 1, 1, 0, 0, 0.000

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 45/45 [00:00<00:00, 1219.85 ops/s]


round:  9
0 [1, 1, 1, 0, 0, 0.00131988525390625, 0.00034999847412109375, 0.004561901092529297]
50 [1, 1, 1, 0, 0, 0.0009388923645019531, 0.00025391578674316406, 0.0034837722778320312]
100 [1, 1, 1, 0, 0, 0.0009169578552246094, 0.000247955322265625, 0.0038268566131591797]
150 [1, 1, 1, 0, 0, 0.0012979507446289062, 0.0003132820129394531, 0.0036411285400390625]
200 [1, 1, 1, 0, 0, 0.0010371208190917969, 0.00027108192443847656, 0.0037050247192382812]
250 [1, 1, 1, 0, 0, 0.0007517337799072266, 0.00020360946655273438, 0.006320953369140625]
300 [1, 1, 1, 0, 0, 0.0008139610290527344, 0.00023603439331054688, 0.003571033477783203]
350 [1, 1, 1, 0, 0, 0.0012249946594238281, 0.00045680999755859375, 0.0042040348052978516]
400 [1, 1, 1, 0, 0, 0.0012431144714355469, 0.0003209114074707031, 0.004034757614135742]
450 [1, 1, 1, 0, 0, 0.0009019374847412109, 0.0002460479736328125, 0.003443002700805664]
500 [1, 1, 1, 0, 0, 0.0007870197296142578, 0.00021123886108398438, 0.0034902095794677734]
550 [1, 1, 1, 0

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 45/45 [00:00<00:00, 965.28 ops/s]


round:  10
0 [1, 1, 1, 0, 0, 0.0014920234680175781, 0.0003352165222167969, 0.003515005111694336]
50 [1, 1, 1, 0, 0, 0.0008368492126464844, 0.0002541542053222656, 0.0036230087280273438]
100 [1, 1, 1, 0, 0, 0.0012459754943847656, 0.0002701282501220703, 0.0035400390625]
150 [1, 1, 1, 0, 0, 0.0009860992431640625, 0.0002868175506591797, 0.0034911632537841797]
200 [1, 1, 1, 0, 0, 0.0008950233459472656, 0.000286102294921875, 0.0035767555236816406]
250 [1, 1, 1, 0, 0, 0.002009153366088867, 0.00043392181396484375, 0.026340723037719727]
300 [1, 1, 1, 0, 0, 0.00140380859375, 0.0002932548522949219, 0.0020668506622314453]
350 [1, 1, 1, 0, 0, 0.0016608238220214844, 0.000324249267578125, 0.0027990341186523438]
400 [1, 1, 1, 0, 0, 0.0009949207305908203, 0.00026106834411621094, 0.003383159637451172]
450 [1, 1, 1, 0, 0, 0.0015728473663330078, 0.0002880096435546875, 0.003470897674560547]
500 [1, 1, 1, 0, 0, 0.0019681453704833984, 0.00031495094299316406, 0.002148151397705078]
550 [1, 1, 1, 0, 0, 0.0010399

In [12]:
valloader = torch.utils.data.DataLoader(val_data, batch_size = 1)

In [13]:
for i, (images, labels) in enumerate(valloader):
    x = images
    y = labels 
    break

In [14]:
traced_model = torch.jit.trace(model, x)
### Original Model
since_1 = time.time()
torch_out = model(x)
inference_time_original = time.time() - since_1

In [15]:
torch_out

tensor([[ -6.6358,   1.2895,   0.4550,  ...,  21.1323,  -5.8902,   4.0097],
        [  0.6449,   6.2741,  34.7097,  ...,  -0.3277,  -1.8668,  -7.1835],
        [ -7.5961,  15.3428,  -2.6259,  ...,   0.8016,   2.4336,  -0.9590],
        ...,
        [ -2.7858,  -5.1429,   0.6817,  ...,  -1.0616,   0.7974,  13.7856],
        [ 22.0681,  -8.9676,   1.9604,  ...,  -7.2898,  -3.2032,   0.6012],
        [ -5.8501,  -7.5255, -12.7759,  ...,  -4.8417,   3.1106,   4.5962]],
       grad_fn=<AddmmBackward>)

In [16]:
torch_out.shape

torch.Size([128, 10])

In [ ]:
y2 = torch.argmax(ort_outs_tensor,1)

In [17]:
onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
load_time_onnx = time.time() - since_1
onnx.checker.check_model(onnx_model)
size_onnx = os.path.getsize(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
#def to_numpy(tensor):
#    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
since_1 = time.time()
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

In [18]:
ort_outs[0]

array([[ -6.635796  ,   1.2895085 ,   0.4549976 , ...,  21.132263  ,
         -5.890247  ,   4.009672  ],
       [  0.64487183,   6.274112  ,  34.709663  , ...,  -0.32768404,
         -1.8667655 ,  -7.18354   ],
       [ -7.5961018 ,  15.342848  ,  -2.6259325 , ...,   0.80155146,
          2.4335592 ,  -0.9590169 ],
       ...,
       [ -2.7858493 ,  -5.142924  ,   0.6817392 , ...,  -1.061623  ,
          0.7973738 ,  13.785639  ],
       [ 22.068138  ,  -8.967614  ,   1.9604057 , ...,  -7.2897606 ,
         -3.2032096 ,   0.6011924 ],
       [ -5.8500614 ,  -7.525514  , -12.775908  , ...,  -4.8417344 ,
          3.110595  ,   4.5961523 ]], dtype=float32)

In [19]:
ort_outs[0].shape

(128, 10)

In [21]:
ort_outs_tensor = torch.from_numpy(ort_outs[0]) 
y2 = torch.argmax(ort_outs_tensor,1)

In [22]:
y2

tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0,
        2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4,
        1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 4, 4, 4, 9, 2,
        5, 4, 7, 6, 7, 9, 0, 5])

In [24]:
y1 = torch.argmax(torch_out,1)
y1

tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0,
        2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4,
        1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 4, 4, 4, 9, 2,
        5, 4, 7, 6, 7, 9, 0, 5])

In [20]:
np.testing.assert_array_equal(to_numpy(torch_out), ort_outs[0])

AssertionError: 
Arrays are not equal

Mismatched elements: 975 / 1280 (76.2%)
Max absolute difference: 7.6293945e-06
Max relative difference: 4.0582905e-05
 x: array([[ -6.635797,   1.289509,   0.454997, ...,  21.132267,  -5.890248,
          4.009673],
       [  0.644872,   6.274111,  34.709663, ...,  -0.327684,  -1.866765,...
 y: array([[ -6.635796,   1.289508,   0.454998, ...,  21.132263,  -5.890247,
          4.009672],
       [  0.644872,   6.274112,  34.709663, ...,  -0.327684,  -1.866765,...

In [17]:
value, pred = torch.max(torch_out,1)
y0 = pred.data.cpu()
#y0 = to_categorical(np.argmax(k_predict, 1), num_classes = 10)
correct_original = np.sum(y0 == y)
accuracy_original = model_scores(y, y0)

TypeError: sum() received an invalid combination of arguments - got (out=NoneType, axis=NoneType, ), but expected one of:
 * (*, torch.dtype dtype)
      didn't match because some of the keywords were incorrect: out, axis
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)


In [16]:
torch_out

tensor([[ -8.2500,  -2.7445,  -0.5753,  ...,  25.1626,  -5.8871,   7.7903],
        [  1.2065,   1.7789,  23.8840,  ...,  -1.4965,   0.7334, -20.8894],
        [ -3.5478,  17.6331,  -5.0308,  ...,  -1.1372,   2.4654,  -2.9282],
        ...,
        [ -1.4990,  -4.2816,  -4.9350,  ...,  -2.6917,  -0.7168,  13.0132],
        [ 26.7344, -17.2551,   0.4890,  ..., -18.8161,   3.8191,  -3.2966],
        [ -4.0466,  -6.6173, -44.3387,  ...,  -6.3366,  11.4678,  14.2239]],
       grad_fn=<AddmmBackward>)

In [20]:
prediction = torch.argmax(torch_out,1)
prediction

tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0,
        2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4,
        1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 9, 4, 4, 9, 2,
        5, 4, 7, 6, 7, 9, 0, 5])

In [22]:
accuracy_ = model_scores(to_numpy(y), to_numpy(prediction))
accuracy_

99.21875

In [41]:
traced_model = torch.jit.trace(model, x)
coreml_model = coremltools.convert(traced_model,inputs=[coremltools.TensorType(shape=x.shape)])

#coreml_model = coremltools.models.MLModel(coreml_path+framework+"/{}/{}-v2.mlmodel".format(model_short_name, model_name))
input_name = coreml_model.get_spec().description.input[0].name
#print(input_name)
#spec = coreml_model.get_spec()
#coreml_model = coremltools.models.MLModel(spec)
name_1 = coreml_model.get_spec().description.input[0].name

output_dict_test = coreml_model.predict({name_1:to_numpy(x)})
####### Mis-classification coreML ######################################
#y3 = torch.argmax(output_dict_test['Identity'],1)

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 45/45 [00:00<00:00, 1465.46 ops/s]


In [48]:
output_tensor = torch.from_numpy(output_dict_test[list(output_dict_test.keys())[0]]) 

In [49]:
y3 = torch.argmax(output_tensor,1)
y3

tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0,
        2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4,
        1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 9, 4, 4, 9, 2,
        5, 4, 7, 6, 7, 9, 0, 5])